In [91]:
# CONDITIONAL PROBABILITY AND PREDICTION MODELING

In [ ]:
#imports
import pandas as pd
from pprint import pprint
import numpy as np
import csv
from datetime import datetime
import re
from matplotlib import pyplot as plt
from collections import defaultdict 
from collections import Counter
import spacy
nlp = spacy.load("en")

In [6]:
# CELL 1: LOAD, CLEAN AND SORT DATA

# load the data
with open("SEPTA_crm_export-2.csv", "r") as file:
    reader = csv.reader(file)
    data = list(reader)

# get only the tickets that have fully populated information (still over 100,000 of them).
NL = []
for lis in data[1:]:
    if len(lis) == 32:
        NL.append(lis)
        
# put into dataframe.
df = pd.DataFrame(NL, columns = data[0])

# create time stamps for the year variable
l = []
for item in df["customer_incident_date_time"]:
    if item:
        # turn the string into a datetime object
        dto = datetime.strptime(item, "%m/%d/%Y %H:%M:%S")
        l.append(dto)
    # if the data is empty, must keep length of dataframe the same
    else:
        l.append(None)
df["customer_incident_date_time"] = l

# sort the dataframe by timestamp. This helped understand our years with most of the info 2013 - 2020
df2 = df.sort_values(by = "customer_incident_date_time", axis = 0)
df3 = df2.reset_index(drop = True)

#check out the dataframe, the first and last several times are irrelevant and may not be used further.
df3

,cs_ticket_number,how_received,contact_id_c,customer_incident_date_time,mode,route,destination,block_train,vehicle,event,...,esms,titleiv,protected_class,created_date,type,code,department,status,group_code,location
0,695025,Web,,0009-01-10 14:39:00,Trolley,101,Media,train,,,...,false,false,false,01/18/2015 05:57:00,Complaint,65-quality-life,Other Depts,Closed,QualityOfLife,Police
1,644376,Web,,0013-01-01 22:58:00,Regional Rail,CHE,Chestnut Hill East,,2760,,...,false,false,false,12/11/2013 15:57:16,Commendation,C1-courteous,Regional Rail,Closed,Employee,CHE
2,639009,Web,,0013-10-22 21:15:00,CCT,,,,CCT bus,,...,false,false,false,10/27/2013 22:46:22,Complaint,26-cct-same-day,CCT,Closed,Service,CCT
3,641783,Web,,0013-11-01 22:05:00,Regional Rail,WIL,,,9243,,...,false,false,false,11/18/2013 19:45:14,Complaint,65-quality-life,Cust Service,Closed,QualityOfLife,Customer Service
4,645464,Web,,0013-12-19 00:30:00,SubEl,,,,,,...,false,false,false,12/19/2013 14:13:11,Complaint,02-job-performance,SubEl,Closed,Employee,SubEl Stations
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110208,765973,Web,,None,1,1,1,1,1,,...,false,false,false,03/31/2016 18:41:19,Commendation,,,Open,,
110209,765702,Web,,None,1,1,1,1,"\+response.write(9822613*9424304)+\""""",,...,false,false,false,03/31/2016 18:17:37,Complaint,,,Open,,
110210,765609,Web,,None,,,1,,,,...,false,false,false,03/31/2016 18:10:05,Travelinfo,,,Open,,
110211,714555,Web,,None,,,,,,,...,false,false,false,06/04/2015 14:54:00,Travelinfo,,,Open,,


In [7]:
# CELL 2:  PREDICTION MODELING: Length of Customer Complaints

# Below are the variables we want to predict (based on the length of a complaint in a customer ticket).
variables = ["ada_related", "detour_related", "weather_related", 
              "request_refund", "esms", "titleiv", "protected_class"]   
print() 
print("What is the likelihood the variable is true if the customer complaint is a certain length?")
for variable in variables:
    print()
    def prob(df3, complaint_len, variable):

        # First we must grab the total number of complaints that are longer than a specified threshold
        conditional_frame = df3[df3["customer_text_description"].str.len() >= complaint_len]
        total = len(conditional_frame)

        # Of that total, see if the variable we are predicting has value "true", for that ticket.
        both_true_frame = conditional_frame[conditional_frame[variable] == "true"]
        both_true = len(both_true_frame)

        likelihood = round(both_true/total, 2)
        
        
        print("If Length:", complaint_len, " Then", variable, likelihood)

    # iterate through the different lengths to apply to the customer complaint
    for i in [100, 300, 500, 750, 1000, 1500, 2000, 2500]:
        prob(df3, i, variable)   


# Analysis:  As the length of the complaint increases, so does the liklihood that the ticket has a purpose 
#...of being a request refund.  This makes sense !  A refund is a big deal and requires action from the...
#... company.  A customer making such a request would not surprisingly have a lot to say. 


What is the likelihood the variable is true if the customer complaint is a certain length?

If Length: 100  Then ada_related 0.01
If Length: 300  Then ada_related 0.02
If Length: 500  Then ada_related 0.02
If Length: 750  Then ada_related 0.02
If Length: 1000  Then ada_related 0.03
If Length: 1500  Then ada_related 0.03
If Length: 2000  Then ada_related 0.03
If Length: 2500  Then ada_related 0.04

If Length: 100  Then detour_related 0.01
If Length: 300  Then detour_related 0.01
If Length: 500  Then detour_related 0.02
If Length: 750  Then detour_related 0.02
If Length: 1000  Then detour_related 0.02
If Length: 1500  Then detour_related 0.01
If Length: 2000  Then detour_related 0.01
If Length: 2500  Then detour_related 0.02

If Length: 100  Then weather_related 0.02
If Length: 300  Then weather_related 0.02
If Length: 500  Then weather_related 0.02
If Length: 750  Then weather_related 0.02
If Length: 1000  Then weather_related 0.02
If Length: 1500  Then weather_related 0.02
If Length: 

In [8]:
# CELL 3: Create Word Count for each Customer Complaint (will be inputs to CELL 4)

# 5 minute Run Time to create 10,000 counter structures

x = datetime.now()

# create word counts for each customer complaint ticket
def count_f(string):
    dd = defaultdict(int)
    doc = nlp(string)
    for token in doc:
        dd[token.text] += 1
    return dd

counters = df3["customer_text_description"][:10000].apply(count_f)

y = datetime.now()

print(y - x)

0:04:39.965007


In [9]:
# CELL 4: PREDICTION MODELING:  Frequency of particular words in a complaint.

# Run Time 1 minute 30 seconds (with the 10,000 word count objects from Cell 3)

# we will check if the following "problem" words are in the customer complaint.
word_set = {"delays", "delayed", "delay", "problem", "problems", "guarantee", "sit", "however",
                   "please", "told", "not", "issue", "issues", "upset", "made", "angry", "harrassment",
                   "affecting"}

# variable we will try to predict.
variable = "ada_related"   

x = datetime.now()

for word in word_set:    
    def prob(df3, key_word, variable, counters):
        first_cond = 0
        sec_cond = 0
        index = 0
        
        # use our counter structure above to see if a key_word exists in a customer complaint.
        for counter in counters:
            total = 0
            index += 1
            for word, count, in counter.items():
                word = word.lower().strip()
                if word in key_word:
                    total = total + count
            
            # when a key_word exists in a complaint, check if our variable of that same ticket is true.
                # (in this case our variable is ada_related)
            if total > 0:
                first_cond += 1
                if df3[variable].iloc[index] == "true":
                    sec_cond += 1
        try:                   
            likelihood = round(sec_cond / first_cond, 2)
            print("If complaint had word:", key_word, "it has", likelihood, "chance to be", variable)
        except ZeroDivisionError:
            print("None were true")

    prob(df3, [word], variable, counters)   
    
y = datetime.now()
print(y - x)

# ANALYSIS:
#customer tickets with words like delayed, problems, and told, had a higher chance of being ADA related.

If complaint had word: ['delayed'] it has 0.02 chance to be ada_related
If complaint had word: ['issues'] it has 0.0 chance to be ada_related
If complaint had word: ['affecting'] it has 0.0 chance to be ada_related
If complaint had word: ['delay'] it has 0.01 chance to be ada_related
If complaint had word: ['made'] it has 0.01 chance to be ada_related
If complaint had word: ['told'] it has 0.02 chance to be ada_related
If complaint had word: ['problem'] it has 0.01 chance to be ada_related
If complaint had word: ['sit'] it has 0.0 chance to be ada_related
If complaint had word: ['not'] it has 0.01 chance to be ada_related
If complaint had word: ['however'] it has 0.01 chance to be ada_related
If complaint had word: ['issue'] it has 0.01 chance to be ada_related
If complaint had word: ['delays'] it has 0.01 chance to be ada_related
If complaint had word: ['upset'] it has 0.0 chance to be ada_related
If complaint had word: ['angry'] it has 0.01 chance to be ada_related
If complaint had w

In [10]:
# CELL 5: Get the Locations for each Customer Complaint (will be inputs to CELL 6)

# variables to predict based on location of ticket
variables = ["ada_related", "detour_related", "weather_related", "request_refund", 
             "esms", "titleiv", "protected_class"]  
        
# this code shows there are 83 TOTAL locations.
locations = []
for location in df3["location"]:
    if location not in locations:
        locations.append(location)

# lets count locations that actually have complaints
locations = Counter()
for typee,location in zip(df3["type"], df["location"]):
    if typee == "Complaint":
        locations[location] = locations[location] + 1
        
loc_l = []
count_l = []
for locs, counts in locations.items():
    # a couple locations were not locations, but sentences.
    if len(locs) < 100 and locs != "" and locs != " ":
        loc_l.append(locs)
        count_l.append(counts)

In [11]:
#CELL 6; PREDICTION MODELING: Location of Ticket

nest_dd = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

#iterate through the customer tickets, check that tickets location, and check the variables we are predicting... 
    #... on that ticket are labeled "true" or "false". Append these items to our nested default dict.
for variable in variables:
    for location, related in zip(df3["location"], df3[variable]):
        # line below makes sure its a good location that has a minimum number of tickets
        if location in loc_l and locations[location] >=14:
            if related == 'true' or related == 'false':
                nest_dd[location][variable]['total'] = nest_dd[location][variable]['total'] + 1
            if related == 'true':
                nest_dd[location][variable]['true'] = nest_dd[location][variable]['true'] + 1
            if related == 'false':
                nest_dd[location][variable]['false'] = nest_dd[location][variable]['false'] + 1
                
pprint(nest_dd)              

defaultdict(<function <lambda> at 0x7fb70d7fb320>,
            {'AIR': defaultdict(<function <lambda>.<locals>.<lambda> at 0x7fb70d8d0290>,
                                {'ada_related': defaultdict(<class 'int'>,
                                                            {'false': 435,
                                                             'total': 440,
                                                             'true': 5}),
                                 'detour_related': defaultdict(<class 'int'>,
                                                               {'false': 440,
                                                                'total': 440}),
                                 'esms': defaultdict(<class 'int'>,
                                                     {'false': 440,
                                                      'total': 440}),
                                 'protected_class': defaultdict(<class 'int'>,
                                        

                                                                       {'false': 72,
                                                                        'total': 72}),
                                            'detour_related': defaultdict(<class 'int'>,
                                                                          {'false': 72,
                                                                           'total': 72}),
                                            'esms': defaultdict(<class 'int'>,
                                                                {'false': 69,
                                                                 'total': 72,
                                                                 'true': 3}),
                                            'protected_class': defaultdict(<class 'int'>,
                                                                           {'false': 72,
                                                                     

                                                                   {'false': 2199,
                                                                    'total': 2201,
                                                                    'true': 2})}),
             'Public/Gov. Affairs': defaultdict(<function <lambda>.<locals>.<lambda> at 0x7fb70d97c170>,
                                                {'ada_related': defaultdict(<class 'int'>,
                                                                            {'false': 112,
                                                                             'total': 116,
                                                                             'true': 4}),
                                                 'detour_related': defaultdict(<class 'int'>,
                                                                               {'false': 116,
                                                                                'total': 116}),

                                {'ada_related': defaultdict(<class 'int'>,
                                                            {'false': 1656,
                                                             'total': 1673,
                                                             'true': 17}),
                                 'detour_related': defaultdict(<class 'int'>,
                                                               {'false': 1673,
                                                                'total': 1673}),
                                 'esms': defaultdict(<class 'int'>,
                                                     {'false': 1673,
                                                      'total': 1673}),
                                 'protected_class': defaultdict(<class 'int'>,
                                                                {'false': 1673,
                                                                 'total': 1673}),
           

In [12]:
# CELL 7: PREDICTION MODELING: Location of Ticket CONTINUED

correlations = []

# now we can iterate through our nested default dictionary in order to count up, based on the location of 
    #... the ticket, how many times a variable is "true".  
for location, dd in nest_dd.items():
    for variable, dd2 in dd.items():
        for t_f_tot, count in list(dd2.items()):
            l = []
            l.append(location)
            l.append(variable)
            
            # calculate our percentage to see how likely a variable is to be true if its from a location.
                # dd2["total"] is the total amount of tickets from a particular location.
                # dd2["true"] is the number of tickets labeled "true" for a particular variable.
            l.append(round(dd2["true"]/dd2["total"], 3))
            t = tuple(l)
            correlations.append(t)

sorted(correlations, key =lambda x: x[2], reverse = True)     

# Lets check out the highest correlations between Location and type (Variable) of the customer ticket.

# If the location of the ticket is from "Sales", "Septa Key Call Center", or "Revenue Operations",
    #... we can see it is much more likely to be a request refund related ticket ! 

[('Sales', 'request_refund', 0.416),
 ('Sales', 'request_refund', 0.416),
 ('Sales', 'request_refund', 0.416),
 ('Septa Key Call Center', 'request_refund', 0.412),
 ('Septa Key Call Center', 'request_refund', 0.412),
 ('Septa Key Call Center', 'request_refund', 0.412),
 ('Revenue Operations', 'request_refund', 0.207),
 ('Revenue Operations', 'request_refund', 0.207),
 ('Revenue Operations', 'request_refund', 0.207),
 ('RR Rev', 'request_refund', 0.194),
 ('RR Rev', 'request_refund', 0.194),
 ('RR Rev', 'request_refund', 0.194),
 ('CCT Eligibility', 'ada_related', 0.12),
 ('CCT Eligibility', 'ada_related', 0.12),
 ('CCT Eligibility', 'ada_related', 0.12),
 ('EM & C', 'weather_related', 0.115),
 ('EM & C', 'weather_related', 0.115),
 ('EM & C', 'weather_related', 0.115),
 ('Wyoming', 'detour_related', 0.065),
 ('Wyoming', 'detour_related', 0.065),
 ('Wyoming', 'detour_related', 0.065),
 ('Control Center', 'weather_related', 0.056),
 ('Control Center', 'weather_related', 0.056),
 ('Contro

In [106]:
#CELL 8: PREDICTION MODELING: Is our ticket ADA_related ?

# filter our data frame to have tickets that are only ada_related (value = "true")
df3_ada = df3[df3["ada_related"] == 'true']

total = len(df3_ada)
def percents(col):
    for key, value in col.value_counts().iteritems():
        # line below calculates how often a particular attribute of another variable exists when that ticket...
            # is also ADA_related.
        percentage = value / total
        return key, round(percentage,2)
        
# using our filtered data frame, call the "percents" function above to find the most common attributes...
    #... of the other variables on the customer ticket.
most_frequent = df3_ada.apply(percents, axis = 0)

correlations = []
bad_keys = ["cs_ticket_number", "customer_incident_date_time", "customer_text_description", "created_date",
            "ada_related"]

# below we will work to sort all of these variables to see which ones have the highest positive correlation
    # with an ADA related variable.
for key, values in most_frequent.iteritems():
    if values[0] == "" or values[0] == " " or key in bad_keys:
        continue
    else:
        l = []
        l.append(key)
        l.append(values[0])
        l.append(values[1])
        t = tuple(l)
        correlations.append(t)
        
sorted(correlations, key=lambda x: x[2], reverse = True)

# When a ticket is ada_related, it has a 47% probability of 'Bus' being its mode of transportation ... 
     #and a 45% probability it came from the 'Surface Trans' department

[('how_received', 'Web', 1.0),
 ('detour_related', 'false', 1.0),
 ('esms', 'false', 1.0),
 ('titleiv', 'false', 1.0),
 ('request_refund', 'false', 0.99),
 ('protected_class', 'false', 0.99),
 ('status', 'Closed', 0.99),
 ('weather_related', 'false', 0.98),
 ('safety_related', 'false', 0.94),
 ('safety_related', 'false', 0.94),
 ('type', 'Complaint', 0.58),
 ('group_code', 'Employee', 0.58),
 ('mode', 'Bus', 0.47),
 ('department', 'Surface Trans', 0.45),
 ('code', '02-job-performance', 0.21),
 ('location', 'Customer Service', 0.11)]